# Part 4:  Batching, Plotting, Saving

# Setup

In [1]:
import numpy as np
import pandas as pd
import napari
import tifffile
import skimage as ski
import scipy.ndimage as ndi
import glob
import cellpose.models as models
import matplotlib.pyplot as plt
import cv2
import dask
import cellpose.models as models
import sutils
import plotly.graph_objs as go
import plotly.express as px

import glob

In [2]:
viewer = napari.Viewer()

In [3]:
models = models.Cellpose(gpu=True, model_type='cyto')

# Batch processing

Frequently you are going to want to process several images, not just a single, and then look at the results.  First we must get a list of all of the files we want to process.

## Getting the list of files

In [4]:
python_notebooks = glob.glob('*.ipynb')
python_notebooks

['0-Download-Images.ipynb',
 '1-Intro.ipynb',
 '2-Segmentation.ipynb',
 '3.1-many_yeast-cellpose.ipynb',
 '3.2-easy-cellpose.ipynb',
 '3.3-two-channel.ipynb',
 '3.4-cellpose_3d.ipynb',
 '3.5-cellpose_training.ipynb',
 '3.6-Homework.ipynb',
 '4-Batching-Plotting.ipynb',
 'EC-Watershed.ipynb']

glob.glob() takes a string (something in '...') and much like in linux, lets you use * to denote anything, and returns a list of all file names that match that string.  In this case it was simple:  it looked for anything that ended with .ipynb in the current directory.

In [6]:
file_names = glob.glob('files/*/*projection.tif')
file_names

['files\\48hpa\\Plate000_Well2_Object4.tif_projection.tif',
 'files\\48hpa\\Plate000_Well2_Object6.tif_projection.tif',
 'files\\48hpa\\Plate000_Well2_Object9.tif_projection.tif',
 'files\\intact\\Plate000_Well1_Object3.tif_projection.tif',
 'files\\intact\\Plate000_Well1_Object4.tif_projection.tif',
 'files\\intact\\Plate000_Well1_Object6.tif_projection.tif']

 In this case, it is looking in the files folder for anything that is a folder ('files/*/') and then looking to see if anything in those sub-folders contains a file whose name ends with 'projection.tif'

## Making a processing pipeline

We can get one of the files from the list just like in a numpy array:  by using [].  We'll just get the first one.

In [7]:
img = ski.io.imread(file_names[4])
img.shape

(3811, 5553, 3)

In [8]:
viewer.layers.clear()
viewer.add_image(img, channel_axis=2)

[<Image layer 'Image' at 0x298e0429ff0>,
 <Image layer 'Image [1]' at 0x298e0459240>,
 <Image layer 'Image [2]' at 0x298e04c6380>]

Here we have spots in channel 0 and a whole worm in channel 2.  We want to find the area of the worm in channel 2 and then find the number of spots in channel 0.

### Processing DAPI

We will just use the same pipeline we did in lesson 2

In [9]:
DAPI = img[:, :, 2]

smoothed = ndi.gaussian_filter(DAPI, 30)
viewer.add_image(smoothed)

thresholded = smoothed > 150
viewer.add_image(thresholded)

labeled_img, object_count = ndi.label(thresholded)
labeled_img = sutils.remove_objects(labeled_img, area_min=1000, area_max=100000000)
viewer.add_labels(labeled_img)

<Labels layer 'labeled_img' at 0x298e04419f0>

### Processing H3p

And a similar pipeline for the spots to lesson 2 (I had to scale the image down by 1000.0 as gaussian_laplace gets buggy with high intensities)

In [10]:
h3p = img[:,:,0]

LoG = -1000.0 * ndi.gaussian_laplace(h3p/1000.0, sigma=8)
viewer.add_image(LoG, colormap='gray', blending='additive', contrast_limits=[0,1.5])

peaks = ski.feature.peak_local_max(LoG, min_distance=3, threshold_abs=0.5)
viewer.add_points(peaks, size=10)

peak_img = np.zeros_like(LoG)
peak_img[peaks[:,0], peaks[:,1]] = 1
viewer.add_image(peak_img, colormap='gray', blending='additive')


<Image layer 'peak_img' at 0x2993223ff40>

Note the switch to **threshold_abs** instead of **threshold_rel**.  This is because we are going to process several images and some of them might have very bright junk that will skew the peak threshold very high and give inconsistent results between images.

### Combining the two

In [11]:
results_dict = ski.measure.regionprops_table(labeled_img, intensity_image=peak_img, properties=['label', 'area', 'mean_intensity'])
results = pd.DataFrame(results_dict)
results['counts'] = results['mean_intensity'] * results['area']
results

,label,area,mean_intensity,counts
0,1,8269367.0,0.000019,156.0


## Looping over all files

It's easy to loop over all of the items in our list of file_names.  Anything we want to run everytime the loop runs has to be indented so python knows what to include in the loop.

In [15]:
for fname in file_names:
    print(fname)
print('Should run only once!')

files\48hpa\Plate000_Well2_Object4.tif_projection.tif
files\48hpa\Plate000_Well2_Object6.tif_projection.tif
files\48hpa\Plate000_Well2_Object9.tif_projection.tif
files\intact\Plate000_Well1_Object3.tif_projection.tif
files\intact\Plate000_Well1_Object4.tif_projection.tif
files\intact\Plate000_Well1_Object6.tif_projection.tif
Should run only once!


Now we combine the two pipelines we made above and puts it into the file loop.

In [16]:
all_results = []

for fname in file_names:
    # Open the file
    print(fname)
    img = ski.io.imread(fname)

    # Segment the whole worm using our DAPI pipeline
    DAPI = img[:, :, 2]
    smoothed = ndi.gaussian_filter(DAPI, 30)
    thresholded = smoothed > 150
    labeled_img, object_count = ndi.label(thresholded)
    labeled_img = sutils.remove_objects(labeled_img, area_min=1000, area_max=100000000)

    # Find the peaks in the H3P channel using our pipeline
    h3p = img[:,:,0]
    LoG = -1000.0 * ndi.gaussian_laplace(h3p/1000.0, sigma=8)
    peaks = ski.feature.peak_local_max(LoG, min_distance=3, threshold_abs=0.5)
    peak_img = np.zeros_like(LoG)
    peak_img[peaks[:,0], peaks[:,1]] = 1

    # Measure the counts in each worm
    results_dict = ski.measure.regionprops_table(labeled_img, intensity_image=peak_img, properties=['label', 'area', 'mean_intensity'])
    results = pd.DataFrame(results_dict)
    results['counts'] = results['mean_intensity'] * results['area']
    
    # Make sure we have the filename in our table
    results['file'] = fname
    all_results.append(results)

    

files\48hpa\Plate000_Well2_Object4.tif_projection.tif
files\48hpa\Plate000_Well2_Object6.tif_projection.tif
files\48hpa\Plate000_Well2_Object9.tif_projection.tif
files\intact\Plate000_Well1_Object3.tif_projection.tif
files\intact\Plate000_Well1_Object4.tif_projection.tif
files\intact\Plate000_Well1_Object6.tif_projection.tif


At this point "all_results" is a python list, that list contains several individual dataframes (tables) from our files.  We want to combine them into a single large dataframe which is easy to do in pandas using concat.


In [19]:
df = pd.concat(all_results)
df

,label,area,mean_intensity,counts,file
0,1,1594637.0,0.000160,255.0,files\48hpa\Plate000_Well2_Object4.tif_project...
0,1,1306072.0,0.000126,165.0,files\48hpa\Plate000_Well2_Object6.tif_project...
0,1,973282.0,0.000153,149.0,files\48hpa\Plate000_Well2_Object9.tif_project...
0,1,5675248.0,0.000020,116.0,files\intact\Plate000_Well1_Object3.tif_projec...
0,1,8269367.0,0.000019,156.0,files\intact\Plate000_Well1_Object4.tif_projec...
0,1,5191331.0,0.000017,89.0,files\intact\Plate000_Well1_Object6.tif_projec...


# Visualizing Results

## Getting our h3p spot density

For this dataset, the value of interest is the number of spots per millimeter^2.  Our areas are in terms of pixels though, so to correct for that we need a conversion factor.  The resolution of these images is 0.78um/pixel.  We will create a new column in the table called 'scaled_ara' to hold this value.

In [20]:
df['scaled_area']=df['area']*0.00078*0.00078

Next we calculate the Density and store it back in the table with a new column.

In [21]:
df['Density'] = df['counts'] / df['scaled_area']
df

,label,area,mean_intensity,counts,file,scaled_area,Density
0,1,1594637.0,0.000160,255.0,files\48hpa\Plate000_Well2_Object4.tif_project...,0.970177,262.838596
0,1,1306072.0,0.000126,165.0,files\48hpa\Plate000_Well2_Object6.tif_project...,0.794614,207.647937
0,1,973282.0,0.000153,149.0,files\48hpa\Plate000_Well2_Object9.tif_project...,0.592145,251.627656
0,1,5675248.0,0.000020,116.0,files\intact\Plate000_Well1_Object3.tif_projec...,3.452821,33.595719
0,1,8269367.0,0.000019,156.0,files\intact\Plate000_Well1_Object4.tif_projec...,5.031083,31.007241
0,1,5191331.0,0.000017,89.0,files\intact\Plate000_Well1_Object6.tif_projec...,3.158406,28.178773


## Plotting with plotly

We have used plotly a bit before, remember all plotly graphs take a table as the first input (our table's name is 'df'), and then x and y arguments for which of the columns of your table you want on those axes.

In [25]:
px.bar(df, x='file', y='Density', width=400)

But usually we will want to aggregate our data in some way.  For instance here we have 6 images, but 3 are from intact animals and 3 are from 48 hours post amputation animals.  We want to create a column to indicate which is which.  Fortunately we can use the folders they came from.  In pandas (kind of like R for Python), we can break up the 'file' column of our table using split.  The '.str' are to tell pandas that we want to treat these things as strings.

In [30]:
df['Timepoint'] = df['file'].str.split('\\').str[1]
df

,label,area,mean_intensity,counts,file,scaled_area,Density,Timepoint
0,1,1594637.0,0.000160,255.0,files\48hpa\Plate000_Well2_Object4.tif_project...,0.970177,262.838596,48hpa
0,1,1306072.0,0.000126,165.0,files\48hpa\Plate000_Well2_Object6.tif_project...,0.794614,207.647937,48hpa
0,1,973282.0,0.000153,149.0,files\48hpa\Plate000_Well2_Object9.tif_project...,0.592145,251.627656,48hpa
0,1,5675248.0,0.000020,116.0,files\intact\Plate000_Well1_Object3.tif_projec...,3.452821,33.595719,intact
0,1,8269367.0,0.000019,156.0,files\intact\Plate000_Well1_Object4.tif_projec...,5.031083,31.007241,intact
0,1,5191331.0,0.000017,89.0,files\intact\Plate000_Well1_Object6.tif_projec...,3.158406,28.178773,intact


Now we can plot the data as replicates.

In [33]:
px.box(df, x='Timepoint', y='Density', points='all', width=400)

It might be useful to track down a particularly aberrant example, so it would be nice to be able to mouse over a point and see which image it came from.  We can do this by adding a hover_data argument to our plotly graph.

In [36]:
px.box(df, x='Timepoint', y='Density', points='all',hover_data=['file'], width=600, title='Density of H3P+ cells')

There are a variety of different plots we can use from plotly, and if you want to learn more about pandas/plotly we will be offering a course on that shortly after this one.

In [71]:
f = px.violin(df, x='Timepoint', y='scaled_area', points='all',hover_data=['file'], title='Worm Area in um^2')
f.write_html('violin.html')
f

# Aggregating

## groupby, agg

What if we wanted a simple bar plot with mean values and standard deviations for the different Timepoints?  First we would need to calculate the two values for our two timepoints.  Pandas makes this pretty easy to do (if a little wordy)

In [45]:
agged_df = df.groupby(['Timepoint'])['Density'].agg(['mean', 'std']).reset_index()
agged_df

,Timepoint,mean,std
0,48hpa,240.704729,29.171646
1,intact,30.927244,2.709359


All dataframes have functions they can access within them, .groupby() is one of them.  .groupby() is really just re-structuring your existing dataframe, it takes a list of arguments that you want to group things over.  We have multiple files per 'Timepoint', but want everything with the same 'Timepoint' to be aggregated together.  df.groupby(['Timepoint])['Density] does this, but only keeps the ['Density] column of our df table.  Note that we could groupby both 'Timepoint' and 'file' by using ['Timepoint', 'file'], but this does not serve any purpose for our experiment as there is only one worm per file (you WILL want to groupby multiple columns in the homework where there are multiple objects per file).

.agg(['mean', 'std']) then looks at the remaining column(s) of our table, and runs functions over the groups.  So it will run 'mean' over all the 'Density' values, but separate them out by 'Timepoint'.

.reset_index() is added at the end just to make things pretty, pandas tries to be cute with indices and it never works well.

## Bar plot with errors

We can plot using the 'error_y' function in px.bar

In [46]:
px.bar(agged_df, x='Timepoint', y='mean', error_y='std', width=400)

# Calculating statistics

In [47]:
from scipy.stats import ttest_ind

In [55]:
df

,label,area,mean_intensity,counts,file,scaled_area,Density,Timepoint
0,1,1594637.0,0.000160,255.0,files\48hpa\Plate000_Well2_Object4.tif_project...,0.970177,262.838596,48hpa
0,1,1306072.0,0.000126,165.0,files\48hpa\Plate000_Well2_Object6.tif_project...,0.794614,207.647937,48hpa
0,1,973282.0,0.000153,149.0,files\48hpa\Plate000_Well2_Object9.tif_project...,0.592145,251.627656,48hpa
0,1,5675248.0,0.000020,116.0,files\intact\Plate000_Well1_Object3.tif_projec...,3.452821,33.595719,intact
0,1,8269367.0,0.000019,156.0,files\intact\Plate000_Well1_Object4.tif_projec...,5.031083,31.007241,intact
0,1,5191331.0,0.000017,89.0,files\intact\Plate000_Well1_Object6.tif_projec...,3.158406,28.178773,intact


In [53]:
group1 = df[df['Timepoint']=='intact'] ['Density'].values
group2 = df[df['Timepoint']=='48hpa'] ['Density'].values

In [54]:
group1

array([33.59571896, 31.00724111, 28.17877315])

In [56]:
group2

array([262.83859581, 207.64793658, 251.62765569])

In [57]:
ttest_ind(group1, group2)

TtestResult(statistic=-12.402049873262643, pvalue=0.00024298768305168787, df=4.0)

# Saving intermediate files

Frequently you will want to save intermediate files so you can look at them later. 

Let's process the first image in our list, but we'll save the filename we are working on as fname just like we did in the loop.

In [28]:
fname = file_names[0]

This is just our processing code from before run on this single 'fname'

In [58]:
img = ski.io.imread(fname)

DAPI = img[:, :, 2]
smoothed = ndi.gaussian_filter(DAPI, 30)
thresholded = smoothed > 150
labeled_img, object_count = ndi.label(thresholded)
labeled_img = sutils.remove_objects(labeled_img, area_min=1000, area_max=100000000)

h3p = img[:,:,0]
LoG = -1000.0 * ndi.gaussian_laplace(h3p/1000.0, sigma=8)
peaks = ski.feature.peak_local_max(LoG, min_distance=3, threshold_abs=0.5)
peak_img = np.zeros_like(LoG)
peak_img[peaks[:,0], peaks[:,1]] = 1

results = pd.DataFrame(ski.measure.regionprops_table(labeled_img, intensity_image=peak_img, properties=['label', 'area', 'mean_intensity']))
results['counts'] = results['mean_intensity'] * results['area']

results['file'] = fname

all_results.append(results)

For this processing pipeline we probably want to export our DAPI image, the label_image it found, our h3p image, the LoG image we used for peak finding, and then the peak_img for where it found the peaks.  To put these all together we can use np.array() which will combine the images along a new dimension.

In [59]:
print(DAPI.shape)
print(labeled_img.shape)
print(h3p.shape)
print(LoG.shape)
print(peak_img.shape)

combined_img = np.array([DAPI, labeled_img, h3p, LoG, peak_img])
print(combined_img.shape)

(2062, 5543)
(2062, 5543)
(2062, 5543)
(2062, 5543)
(2062, 5543)
(5, 2062, 5543)


In [60]:
viewer.layers.clear()
viewer.add_image(combined_img, channel_axis=0)

[<Image layer 'Image' at 0x298e0930af0>,
 <Image layer 'Image [1]' at 0x298cbff1f00>,
 <Image layer 'Image [2]' at 0x298cc46ab00>,
 <Image layer 'Image [3]' at 0x299323eba00>,
 <Image layer 'Image [4]' at 0x2993310ccd0>]

If we want the image to be readable by ImageJ (which is convenient for dragging and dropping), then we need to convert it do single point floating.

In [61]:
combined_img = combined_img.astype(np.single)

We also need a name to save the file as, we are going to be lazy and just add an "f" to the end, making it *.tiff (meaning that our glob.glob() will not find it if we run things again).

In [62]:
output_filename = fname + 'f'
output_filename

'files\\intact\\Plate000_Well1_Object6.tif_projection.tiff'

Then we use ski.io.imsave() to save the file, we provide the imagej=True argument to make it readable by ImageJ, and the metadata={'axes':'CYX'} so that ImageJ knows what order the axes are in.

In [63]:
ski.io.imsave(output_filename, combined_img,  imagej=True, metadata={'axes': 'CYX'})

Now let's put it back into our loop and run it on all of the files.

In [64]:
all_results = []

for fname in file_names:
    print(fname)
    img = ski.io.imread(fname)

    DAPI = img[:, :, 2]
    smoothed = ndi.gaussian_filter(DAPI, 30)
    thresholded = smoothed > 150
    labeled_img, object_count = ndi.label(thresholded)
    labeled_img = sutils.remove_objects(labeled_img, area_min=1000, area_max=100000000)

    h3p = img[:,:,0]
    LoG = -1000.0 * ndi.gaussian_laplace(h3p/1000.0, sigma=8)
    peaks = ski.feature.peak_local_max(LoG, min_distance=3, threshold_abs=0.5)
    peak_img = np.zeros_like(LoG)
    peak_img[peaks[:,0], peaks[:,1]] = 1

    results = pd.DataFrame(ski.measure.regionprops_table(labeled_img, intensity_image=peak_img, properties=['label', 'area', 'mean_intensity']))
    results['counts'] = results['mean_intensity'] * results['area']
    
    results['file'] = fname
    all_results.append(results)

    # New stuff for saving our intermediate files
    combined_img = np.array([DAPI, labeled_img, h3p, LoG, peak_img])
    combined_img = combined_img.astype(np.single)
    output_filename = fname + 'f'
    ski.io.imsave(output_filename, combined_img,  imagej=True, metadata={'axes': 'CYX'})

    

files\48hpa\Plate000_Well2_Object4.tif_projection.tif
files\48hpa\Plate000_Well2_Object6.tif_projection.tif
files\48hpa\Plate000_Well2_Object9.tif_projection.tif
files\intact\Plate000_Well1_Object3.tif_projection.tif
files\intact\Plate000_Well1_Object4.tif_projection.tif
files\intact\Plate000_Well1_Object6.tif_projection.tif


# Interactive Plots

Sometimes it is nice to be able to easily interact with our data.

This is relatively easy to do with plotly and napari.  If df['file] contains the path to our image we can write the function to open up the image when we click on the data point in the plot.

In [66]:
viewer = napari.Viewer()

In [67]:


f=go.FigureWidget(
    px.violin(df, x='Timepoint', y='Density', points='all',hover_data=['file'], width=600, title='H3p Density')
    )

def click_fn(trace, points, state):
    
    if (len(points.point_inds)>0):
        fname = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(fname)
        img = ski.io.imread(fname)
        viewer.layers.clear()
        viewer.add_image(img, channel_axis=-1)
 
for a in f.data:
    a.on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'box': {'visible': False},
              'customdata': array([['files\\48hpa\\Plate000_Well2_Object4.tif_projection.tif'],
                                   ['files\\48hpa\\Plate000_Well2_Object6.tif_projection.tif'],
                                   ['files\\48hpa\\Plate000_Well2_Object9.tif_projection.tif'],
                                   ['files\\intact\\Plate000_Well1_Object3.tif_projection.tif'],
                                   ['files\\intact\\Plate000_Well1_Object4.tif_projection.tif'],
                                   ['files\\intact\\Plate000_Well1_Object6.tif_projection.tif']],
                                  dtype=object),
              'hovertemplate': 'Timepoint=%{x}<br>Density=%{y}<br>file=%{customdata[0]}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa'},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v'

files\48hpa\Plate000_Well2_Object4.tif_projection.tif


files\intact\Plate000_Well1_Object3.tif_projection.tif
files\intact\Plate000_Well1_Object6.tif_projection.tif


Or we can have it open up the processed image that we just created, since all we did was add an 'f' to the end of the file.

In [68]:


f=go.FigureWidget(
    px.violin(df, x='Timepoint', y='Density', points='all',hover_data=['file'], width=600, title='H3p Density (in spots per mm^2)')
    )

def click_fn(trace, points, state):
    
    if (len(points.point_inds)>0):
        fname = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        fname = fname + 'f'
        print(fname)
        img = ski.io.imread(fname)
        viewer.layers.clear()
        viewer.add_image(img, channel_axis=0)
 
for a in f.data:
    a.on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'box': {'visible': False},
              'customdata': array([['files\\48hpa\\Plate000_Well2_Object4.tif_projection.tif'],
                                   ['files\\48hpa\\Plate000_Well2_Object6.tif_projection.tif'],
                                   ['files\\48hpa\\Plate000_Well2_Object9.tif_projection.tif'],
                                   ['files\\intact\\Plate000_Well1_Object3.tif_projection.tif'],
                                   ['files\\intact\\Plate000_Well1_Object4.tif_projection.tif'],
                                   ['files\\intact\\Plate000_Well1_Object6.tif_projection.tif']],
                                  dtype=object),
              'hovertemplate': 'Timepoint=%{x}<br>Density=%{y}<br>file=%{customdata[0]}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa'},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v'

files\48hpa\Plate000_Well2_Object4.tif_projection.tiff


# Homework

## Part 1:  Find in situ spots in image

Choose one of the files in files/batch_homework.  Use our peak finding tricks to find what you think are real peaks in the first channel of the image.  

In [37]:
fname = 'files/batch_homework/chamber1KLF5in488SOX8in647-1.tif'
img = ski.io.imread(fname)
img.shape

(3072, 3072, 3)

In [38]:
viewer.layers.clear()
viewer.add_image(img, channel_axis=2)

[<Image layer 'Image' at 0x12ba361eb30>,
 <Image layer 'Image [1]' at 0x12ba3756950>,
 <Image layer 'Image [2]' at 0x12b95ae7d30>]

In [39]:
h3p = img[:,:,0]
LoG = -1000.0 * ndi.gaussian_laplace(h3p/1000.0, sigma=5)
peaks = ski.feature.peak_local_max(LoG, min_distance=3, threshold_abs=5.5, num_peaks=1000000)  # I found threshold_abs needed to be above 5.5, but this is true only if a similar sigma is used for LoG
#peak_img = np.zeros_like(LoG)
#peak_img[peaks[:,0], peaks[:,1]] = 1

viewer.add_image(LoG, colormap='gray', blending='additive', contrast_limits=[0,1.5])
viewer.add_points(peaks, size=5)

<Points layer 'peaks' at 0x12b96131660>

If you chose an image from "chamber1", try your settings on an image from "chamber2", if the settings don't work well, adjust them until they work well for both.

In [41]:
fnameB = 'files/batch_homework/chamber2KLF5in488SOX8in647-1.tif'
img = ski.io.imread(fnameB)
img.shape

(3072, 3072, 3)

In [42]:
viewer.layers.clear()
viewer.add_image(img, channel_axis=2)

[<Image layer 'Image' at 0x12b9618f460>,
 <Image layer 'Image [1]' at 0x12b95c35660>,
 <Image layer 'Image [2]' at 0x12b95bad7e0>]

In [43]:
h3p = img[:,:,0]
LoG = -1000.0 * ndi.gaussian_laplace(h3p/1000.0, sigma=5)
peaks = ski.feature.peak_local_max(LoG, min_distance=3, threshold_abs=5.5, num_peaks=1000000)  # I found threshold_abs needed to be above 5.5, but this is true only if a similar sigma is used for LoG
peak_img = np.zeros_like(LoG)
peak_img[peaks[:,0], peaks[:,1]] = 1

viewer.add_image(LoG, colormap='gray', blending='additive', contrast_limits=[0,1.5])
viewer.add_points(peaks, size=5)

<Points layer 'peaks' at 0x12b960c7ac0>

## Part 2:  Segment cells using cellpose

Use cellpose to segment the cells in one of the images (should be able to use last week's homework to do so).

In [45]:
model = models.Cellpose(gpu=True, model_type='cyto')

In [46]:
cells, flows, styles, diams = model.eval(img, diameter=300, channels=[2,3], channel_axis=2, cellprob_threshold=-2, flow_threshold=1.0)
filtered_cells = sutils.remove_objects(cells, 50000, 10000000000)
shrunk_cells = sutils.shrink_labels(filtered_cells, shrinkage=40)
viewer.add_labels(shrunk_cells, name='Shrunk cells')

<Labels layer 'Shrunk cells' at 0x12b9616add0>

## Part 3:  Create a table of spots per cell in an image

Use regionprops to calculate the number of spots per cell, and then pd.DataFrame to make a dataframe out of it.

In [47]:
results = pd.DataFrame(ski.measure.regionprops_table(shrunk_cells, intensity_image=peak_img, properties=['label', 'area', 'mean_intensity']))
results['counts'] = results['mean_intensity'] * results['area']
results.head()

,label,area,mean_intensity,counts
0,1,57769.0,0.000260,15.0
1,2,173090.0,0.000023,4.0
2,4,64750.0,0.000031,2.0
3,6,107958.0,0.000389,42.0
4,21,115401.0,0.000087,10.0


## Part 4:  Process all 8 images using glob

Use glob and a for loop to apply your in situ spot finder, cell segmenter, and table maker to all cells, and put together a master dataframe at the end that contains all the values.

MAKE SURE you are reasonably happy with your values above, as this will take a while to run.

In [48]:
all_results = []
for f in glob.glob('files/batch_homework/*'):
    img = ski.io.imread(f)
    h3p = img[:,:,0]
    LoG = -1000.0 * ndi.gaussian_laplace(h3p/1000.0, sigma=5)
    peaks = ski.feature.peak_local_max(LoG, min_distance=3, threshold_abs=5.5, num_peaks=1000000)  # I found threshold_abs needed to be above 5.5, but this is true only if a similar sigma is used for LoG
    peak_img = np.zeros_like(LoG)
    peak_img[peaks[:,0], peaks[:,1]] = 1

    cells, flows, styles, diams = model.eval(img, diameter=300, channels=[2,3], channel_axis=2, cellprob_threshold=-2, flow_threshold=1.0)
    filtered_cells = sutils.remove_objects(cells, 50000, 10000000000)
    shrunk_cells = sutils.shrink_labels(filtered_cells, shrinkage=40)

    results = pd.DataFrame(ski.measure.regionprops_table(shrunk_cells, intensity_image=peak_img, properties=['label', 'area', 'mean_intensity']))
    results['counts'] = results['mean_intensity'] * results['area']

    results['file'] = f
    all_results.append(results)

In [49]:
df = pd.concat(all_results)

## Part 5:  Plot results

First create two new columns based on df['file']:  one that has just the filename without the path (eg we just want chamber1KLF5in488SOX8in647-1.tif).  The second one should just contain which chamber.  You will want to make use of df['file'].str.split('???').str[???] for both of these.

In [50]:
df['short_file'] = df['file'].str.split('\\').str[-1]
df['chamber'] = df['short_file'].str.split('chamber').str[1].str[0]

Make violin plots for the spots/area of all 8 files using the shortened name you created above, make sure you use points='all to visualize the individual cells.

In [51]:
px.violin(df, x='short_file', y='mean_intensity', points='all', width=1200)

## Part 6:  Aggregating

There is pretty clearly a difference between the two, but it is not being captured here.  The likely problem is that transfection sucks and should never be used for quantitative experiments:  only some of the cells appear to be different in some way.  One solution is to assume that only 20% of cells are transfected, and then take the most dense 20% as an upper bound.

.agg() takes names like 'mean' and 'median', but will also take whatever function we give it.  I have defined a function that returns the 80th percentile below, use this in .agg() to calculate the 80th percentile of each file's spot density.

In [52]:
## DO NOT DELETE IN STUDENT VERSION ###

def ptile(x):
    return np.percentile(x, 80)

In [53]:
agged = df.groupby(['chamber', 'short_file'])['mean_intensity'].agg(ptile).reset_index()
agged

,chamber,short_file,mean_intensity
0,1,chamber1KLF5in488SOX8in647-1.tif,0.000108
1,1,chamber1KLF5in488SOX8in647-2.tif,0.000092
2,1,chamber1KLF5in488SOX8in647-3.tif,0.000082
3,1,chamber1KLF5in488SOX8in647-4.tif,0.000107
4,2,chamber2KLF5in488SOX8in647-1.tif,0.000250
5,2,chamber2KLF5in488SOX8in647-2.tif,0.000203
6,2,chamber2KLF5in488SOX8in647-3.tif,0.000332
7,2,chamber2KLF5in488SOX8in647-4.tif,0.000289


Now plot the results of the agged version in a box or violin plot.

In [54]:
px.box(agged, x='chamber', y='mean_intensity', points='all', width=600)

## Part 7: Statistics

Calculate a p-value for our experiment, assuming each image is an independent experiment.

In [55]:
group1 = agged[agged['chamber']=='1']['mean_intensity']
group2 = agged[agged['chamber']=='2']['mean_intensity']

ttest_ind(group1, group2)


TtestResult(statistic=-6.100448558151106, pvalue=0.0008839080892721419, df=6.0)